![header](../img/header.jpg)

# Exercise 7: MODFLOW 6

The purpose of this exercise is to convert the RIV and lake CHD packages to SFR and LAK packages, respectively, and add a mover package to transfer water between the SFR and LAK packages. We will also use a time series file to define SFR inflows  and add LAK and SFR observations.

In [ ]:
import os
import sys
import numpy as np
import flopy
import matplotlib as mpl
import matplotlib.pyplot as plt

import config

print(sys.version)
print('python executable: {}'.format(sys.executable))
print('numpy version: {}'.format(np.__version__))
print('matplotlib version: {}'.format(mpl.__version__))
print('flopy version: {}'.format(flopy.__version__))

In [ ]:
# load the existing model
model_ws = './ex06-data'
s = flopy.mf6.MFSimulation().load(sim_ws=model_ws, exe_name=config.mf6exe)
s.simulation_data.max_columns_of_data = 23

In [ ]:
# get list of files in the ex07-data directory
ex07ddir = './ex07-data'
os.listdir(ex07ddir)

In [ ]:
# change work space
model_ws = './ex07-completed'
s.set_sim_path(model_ws)

In [ ]:
# get the groundwater model
gwf = s.get_model('ex06')

In [ ]:
# get the river location data
riv = gwf.get_package('RIV-1')
rivspd = riv.stress_period_data.get_data(key=0)
rivspd.dtype

In [ ]:
# get rid of the existing river package
gwf.remove_package('RIV')

### create the SFR model

In [ ]:
# read sfr package data
sfrpd = np.genfromtxt('./ex07-data/sfr-packagedata.dat', names=True)
sfrpd.dtype

In [ ]:
# create a default dtype
sfrpackagedata = flopy.mf6.ModflowGwfsfr.packagedata.empty(gwf, boundnames=True, maxbound=rivspd.shape[0])
sfrpackagedata.dtype

In [ ]:
# fill package data
for name in sfrpackagedata.dtype.names :
    if name in rivspd.dtype.names:
        sfrpackagedata[name] = rivspd[name]
for name in sfrpackagedata.dtype.names:
    if name in sfrpd.dtype.names:
        sfrpackagedata[name] = sfrpd[name]
sfrpackagedata['boundnames'] = rivspd['boundname']    
sfrpackagedata    

In [ ]:
type(sfrpackagedata['cellid'][0])

In [ ]:
# read sfr connection data
with open('ex07-data/sfr-connectiondata.dat') as f: 
    lines = f.readlines()
sfrconnectiondata = []
for line in lines:
    t = line.split()
    c = []
    for v in t:
        i = int(v)
        c.append(i)
    sfrconnectiondata.append(c)

In [ ]:
# load inflow timeseries data
inflow_data = np.loadtxt('./ex07-data/sfr_timeseries.dat')
inflow_data = list(map(tuple, inflow_data))

In [ ]:
# create sfr inflows
sfrperioddata = {0: [[0, 'inflow', 'inflow_rate']]}
time_series_dict = {'filename': 'sfr-inflow-rates.ts',
                    'timeseries': inflow_data,
                    'time_series_namerecord': 'inflow_rate',
                    'interpolation_methodrecord': 'linear'}

In [ ]:
# add the sfr package to the model
sfr = flopy.mf6.ModflowGwfsfr(gwf, stage_filerecord='ex06.sfr.stage.bin', budget_filerecord='ex06.sfr.cbc', 
                              mover=True, pname='SFR-1',
                              unit_conversion=128390.00, 
                              boundnames=True, nreaches=38,
                              packagedata=sfrpackagedata, 
                              connectiondata=sfrconnectiondata,
                              perioddata=sfrperioddata,
                              timeseries=time_series_dict)

In [ ]:
sfrobsname = 'ex06.sfr.obs'
sfr_obs = [('SFR06-S', 'STAGE', 5), ('SFR06-Q', 'DOWNSTREAM-FLOW', 5),
           ('SFR07-S', 'STAGE', 6), ('SFR07-Q', 'DOWNSTREAM-FLOW', 6),
           ('SFR38-S', 'STAGE', 37), ('SFR38-Q', 'DOWNSTREAM-FLOW', 37),
           ('SEG01', 'SFR', 'SEG1'), ('SEG02', 'SFR', 'SEG2'),
           ('SEG03', 'SFR', 'SEG3'), ('SEG04', 'SFR', 'SEG4')]
sfr.obs.initialize(filename=sfrobsname, continuous={sfrobsname + '.csv': sfr_obs})

In [ ]:
#sfr.ts.initialize(filename='sfr-inflow-rates.ts',
#                  timeseries=inflow_data,
#                  time_series_namerecord=['inflow_rate'],
#                  interpolation_methodrecord=['linear'])

### Create the Lake package

In [ ]:
# get the lake chd data
lakchd = gwf.get_package('LAK-1')
chdspd = lakchd.stress_period_data.get_data(key=0)
chdloc = chdspd['cellid']
chdloc

In [ ]:
# remove the lake chd file
gwf.remove_package('LAK-1')

In [ ]:
# read lak package data
dtype = [('lakeno', np.int32), ('iconn', np.int32), 
         ('k', np.int32), ('i', np.int32), ('j', np.int32), 
         ('claktype', '|U20'), ('bedleak', np.float), ('belev', np.float), 
         ('telev', np.float), ('connlen', np.float), ('connwidth', np.float)]
lakpd = np.genfromtxt('./ex07-data/lak-connectiondata.dat', dtype=dtype)
lakpd.dtype

In [ ]:
# create cellid
arr = np.column_stack((lakpd['k'], lakpd['i'], lakpd['j']))
cellid = tuple(map(tuple, arr))
len(cellid)

In [ ]:
# create lakeconnectiondata
lakeconnectiondata = flopy.mf6.ModflowGwflak.connectiondata.empty(gwf, maxbound=len(cellid))
for name in lakeconnectiondata.dtype.names:
    if name == 'cellid':
        lakeconnectiondata[name] = cellid
    else:
        lakeconnectiondata[name] = lakpd[name]
lakeconnectiondata

In [ ]:
# get the unique lake numbers
uniquelakes = np.unique(lakpd['lakeno'])
uniquelakes

In [ ]:
nlakecon = np.zeros(uniquelakes.shape, dtype=np.int)
for lak in lakpd['lakeno']:
    for idx, lid in enumerate(uniquelakes):
        if lid == lak:
            nlakecon[idx] += 1
nlakecon

In [ ]:
lakpackagedata = [[0, 44., nlakecon[0], 'lake1'],
                  [1, 35.2, nlakecon[1], 'lake2']]

In [ ]:
# create outlet data
outlet=[[0, 0, 0, 'MANNING', 44.5, 5.000000, 0.03,  0.2187500E-02]]

In [ ]:
# create the lake package
lak = flopy.mf6.ModflowGwflak(gwf, pname='LAK-1', time_conversion=86400.000, 
                              mover=True, boundnames=True, 
                              nlakes=2, noutlets=1,
                              outlets=outlet, 
                              packagedata=lakpackagedata, 
                              connectiondata=lakeconnectiondata)

In [ ]:
# create lake observations
lakobsname = 'ex06.lak.obs'
lak_obs = [('LAK1-S', 'STAGE', 1),
           ('LAK2-S', 'STAGE', 2),
           ('LAK1-Q', 'LAK', 'LAKE1'), 
           ('LAK2-Q', 'LAK', 'LAKE2')]
#lak.obs.filename = lakobsname
#lak.obs.continuous = {lakobsname+'.csv': lak_obs}
lak.obs.initialize(filename=lakobsname, continuous={lakobsname+'.csv': lak_obs})

In [ ]:
gwf

### reset idomain using lak chd locations

In [ ]:
idomain = gwf.dis.idomain.array
for loc in chdloc:
    idomain[loc] = 0
gwf.dis.idomain.set_data(idomain[0], layer=0, multiplier=[1])

### Create the MVR package

In [ ]:
mvrperioddata = [['SFR-1', 5, 'LAK-1', 0, 'FACTOR',  1.],
                 ['LAK-1', 0, 'SFR-1', 6, 'FACTOR',  1.]]

In [ ]:
mvr = flopy.mf6.ModflowGwfmvr(gwf, maxmvr=2, maxpackages=2, packages=['SFR-1', 'LAK-1'], perioddata=mvrperioddata)

### Write the MODFLOW 6 files

In [ ]:
s.write_simulation()
s.run_simulation()

In [ ]:
# load sfr and lak obs
sfrobs = np.genfromtxt(os.path.join(model_ws, 'ex06.sfr.obs.csv'), delimiter=',', names=True)
lakobs = np.genfromtxt(os.path.join(model_ws, 'ex06.lak.obs.csv'), delimiter=',', names=True)
lakobs.dtype

In [ ]:
# create mapping array for lake data
lakeconn = gwf.lak.connectiondata.get_data()
lakmap = {0: [], 1: []}
for v in lakeconn:
    if v['claktype'].upper() == 'VERTICAL':
        cid = v['cellid']
        lakmap[v['lakeno']].append((0, cid[1], cid[2]))

In [ ]:
def sub_lake(h, ls, tmap):
    for key, value in tmap.items():
        s = ls[key]
        for loc in value:
            h[loc] = s
    return h    

In [ ]:
# retrieve the heads
hobj = flopy.utils.HeadFile(os.path.join(model_ws, 'ex06.hds'))
times= hobj.get_times()

In [ ]:
h = hobj.get_data(totim=times[0])
ls = [lakobs['LAK1S'][0], lakobs['LAK2S'][0]]
h = sub_lake(h, ls, lakmap)
pmv = flopy.plot.PlotMapView(gwf)
pmv.plot_array(h, masked_values=[1e+30])
c = pmv.contour_array(h, masked_values=[1e+30], colors='white', levels=np.arange(30, 50, 2))
plt.clabel(c, fmt='%3d')
plt.title('time = {}'.format(times[0]));

In [ ]:
h = hobj.get_data(totim=times[-1])
ls = [lakobs['LAK1S'][-1], lakobs['LAK2S'][-1]]
h = sub_lake(h, ls, lakmap)
pmv = flopy.plot.PlotMapView(gwf)
pmv.plot_array(h, masked_values=[1e+30])
c = pmv.contour_array(h, masked_values=[1e+30], colors='white', levels=np.arange(30, 50, 2))
plt.clabel(c, fmt='%3d')
plt.title('time = {}'.format(times[-1]));

In [ ]:
# plot the sfr stage results
names = [name for name in sfrobs.dtype.names[1:] if name[-1] == 'S']
f, axes = plt.subplots(nrows=1, ncols=len(names), figsize=(len(names)*5, 5))
axes = axes.flatten()
for idx, name in enumerate(names):
    axes[idx].plot(sfrobs['time'], sfrobs[name], marker='.', label=name)
    axes[idx].legend();

In [ ]:
# plot the sfr Q results
names = [name for name in sfrobs.dtype.names[1:] if name[-1] == 'Q']
f, axes = plt.subplots(nrows=1, ncols=len(names), figsize=(len(names)*5, 5))
axes = axes.flatten()
for idx, name in enumerate(names):
    axes[idx].plot(sfrobs['time'], sfrobs[name], marker='.', label=name)
    axes[idx].legend();

In [ ]:
# plot the lake results
names = [name for name in lakobs.dtype.names[1:] if name[-1] == 'S']
f, axes = plt.subplots(nrows=1, ncols=len(names), figsize=(len(names)*5, 5))
axes = axes.flatten()
for idx, name in enumerate(names):
    axes[idx].plot(lakobs['time'], lakobs[name], marker='.', label=name)
    axes[idx].legend();